# Taller 2: Aprendizaje Supervisado – Modelos de Clasificación
Integrantes: Zada Riquelme y Vania Reyes.

------------------------------------------------------------------------------------------------------------------------

# Actividad 1

Para el preprocesamiento de datos, llevamos a cabo una limpieza de columnas y rellenamos los datos perdidos con ceros, esta última decisión la tomamos al ser que la columna INS_1SEM e INS_2SEM proporciona información de las asignaturas que se tomaron por cada semestre cada alumno, lo que nos llevó a conjeturar que los promedios que se encontraban vacíos es porque el estudiante no los inscribió por lo que nos pareció más prudente reemplazarlos por ceros que por la media, la cual era otra de nuestras opciones.
A continuación mencionamos las columnas que determinamos que no son relevantes para este objetivo del taller: ACTF_1SEM_R, ACTF_2SEM_R, descripcion_situacion_egreso_postulante, nombre_secretaria_admision, descripcion_jefe_familia, descripcion_nivel_educacion_padre, descripcion_nivel_educacion_madre, descipcion_tipo_organismo_trabajan_padre, descripcion_tipo_organismo_trabajan_madre, descripcion_ocupacion_principal_padre, descripcion_ocupacion_principal_madre, descripcion_rama_actividad_padre, descripcion_rama_actividad_madre, cuantos_trabajan_grupos_familiar, cuantos_estudian_grupo_familiar, cuantos_estudian_grupo_pre_basica, cuantos_estudian_grupo_media_1_3, cuantos_estudian_grupo_media_4, cuantos_estudian_grupo_otras.

# Actividad 2

En este caso, se ha elegido la estrategia de validación cruzada K-fold, que consiste en dividir el conjunto de datos en 'k' particiones y entrenar el modelo en 'k-1' particiones, mientras se valida en la partición restante. Esta estrategia ayuda a reducir el riesgo de overfitting y a obtener una estimación más precisa del desempeño del modelo.

En el código proporcionado, se ha implementado la validación cruzada K-fold con 'k' igual a 5. Se han llevado a cabo las siguientes acciones: se han cargado los datos desde un archivo CSV, se han seleccionado las columnas numéricas y categóricas, se han definido dos pipelines de preprocesamiento, se ha aplicado el transformador de columnas a los datos, se ha codificado la etiqueta y se han convertido los datos a float32. Luego, se ha definido una función para crear el modelo, se ha establecido el número de pliegues y se ha inicializado el generador de particiones KFold. Finalmente, se ha entrenado el modelo en cada pliegue y se ha calculado la precisión promedio en validación cruzada K-fold.

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

X = datos.iloc[:, :-1]
y = datos.iloc[:, -1]
X = X.drop('Id', axis=1)

columnas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = X.select_dtypes(include=['object']).columns

pipeline_numerico = Pipeline([
    ('imputador', SimpleImputer(strategy='median')),
    ('escalador', StandardScaler())
])

pipeline_categorico = Pipeline([
    ('imputador', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocesador = ColumnTransformer([
    ('num', pipeline_numerico, columnas_numericas),
    ('cat', pipeline_categorico, columnas_categoricas)
])

X_procesado = preprocesador.fit_transform(X)
y = pd.get_dummies(y, drop_first=True).values
X_procesado = X_procesado.astype('float32')
y = y.astype('float32')

def crear_modelo():
    modelo = Sequential()
    modelo.add(Dense(512, activation='relu', input_shape=(X_procesado.shape[1],)))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(512, activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(256, activation='relu'))
    modelo.add(Dense(1, activation='sigmoid'))
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

resultados = []

for fold_index, (train_index, val_index) in enumerate(kf.split(X_procesado, y)):
    print(f'Entrenando fold {fold_index + 1}/{num_folds}...')
    
    X_train, X_val = X_procesado[train_index], X_procesado[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    modelo = crear_modelo()
    
    parada_temprana = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
    
    historia = modelo.fit(X_train, y_train, epochs=200, batch_size=64,
                          validation_data=(X_val, y_val), callbacks=[parada_temprana], verbose=1)
    
    resultados.append(historia.history['val_accuracy'][-1])

resultados = np.array(resultados)
print(f'Precisión promedio en validación cruzada K-fold: {np.mean(resultados):.4f} ± {np.std(resultados):.4f}')


Entrenando fold 1/5...
Epoch 1/200


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6626 - loss: 0.6004 - val_accuracy: 0.7219 - val_loss: 0.5286
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7544 - loss: 0.5064 - val_accuracy: 0.7185 - val_loss: 0.5249
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7464 - loss: 0.4970 - val_accuracy: 0.7252 - val_loss: 0.5247
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7638 - loss: 0.4765 - val_accuracy: 0.7334 - val_loss: 0.5188
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7716 - loss: 0.4639 - val_accuracy: 0.7384 - val_loss: 0.5368
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7954 - loss: 0.4326 - val_accuracy: 0.7285 - val_loss: 0.5303
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8146 - loss: 0.4061 - val_accuracy: 0.7285 - val_loss: 0.5438
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8312 - loss: 0.3743 - val_accuracy: 0.7185 - val_l

# Actividad 3

Se identifican dos tipos de modelos adecuados: Random Forest y SVM. Para cada modelo, se seleccionan al menos dos hiperparámetros que se pueden configurar para producir variantes del modelo. Luego, se ejecuta un proceso de búsqueda del mejor clasificador utilizando los dos tipos de modelos y con al menos dos parámetros o variantes por cada uno. Finalmente, se selecciona el modelo con mejor expectativa de ofrecer un buen resultado.

En el código, se definen los modelos y sus parámetros, se ejecuta la búsqueda del mejor clasificador utilizando GridSearchCV y se selecciona el modelo con mejor precisión. Los resultados muestran que el modelo Random Forest con max_depth=None y n_estimators=200 es el mejor modelo, con una precisión del 73.18% en el conjunto de prueba.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

modelos = {
    'RandomForest': {
        'modelo': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20]
        }
    },
    'SVM': {
        'modelo': SVC(random_state=42),
        'params': {
            'kernel': ['linear', 'rbf'],
            'C': [1, 10, 100]
        }
    }
}

def encontrar_mejor_modelo(modelo, params, X_entrenamiento, y_entrenamiento, X_prueba, y_prueba):
    y_entrenamiento = np.ravel(y_entrenamiento)
    y_prueba = np.ravel(y_prueba)
    
    busqueda_grilla = GridSearchCV(estimator=modelo, param_grid=params, cv=3, scoring='accuracy', verbose=1)
    busqueda_grilla.fit(X_entrenamiento, y_entrenamiento)
    
    mejor_modelo = busqueda_grilla.best_estimator_
    mejores_parametros = busqueda_grilla.best_params_
    
    y_pred = mejor_modelo.predict(X_prueba)
    precision = accuracy_score(y_prueba, y_pred)
    
    return mejor_modelo, mejores_parametros, precision

resultados = {}
for nombre_modelo, config in modelos.items():
    mejor_modelo, mejores_parametros, precision = encontrar_mejor_modelo(config['modelo'], config['params'], X_entrenamiento, y_entrenamiento, X_prueba, y_prueba)
    
    resultados[nombre_modelo] = {
        'mejor_modelo': mejor_modelo,
        'mejores_parametros': mejores_parametros,
        'precision': precision
    }
    
mejor_modelo_nombre = max(resultados, key=lambda x: resultados[x]['precision'])
mejor_modelo = resultados[mejor_modelo_nombre]['mejor_modelo']
mejores_parametros = resultados[mejor_modelo_nombre]['mejores_parametros']
mejor_precision = resultados[mejor_modelo_nombre]['precision']

print(f"Mejor modelo en general: {mejor_modelo_nombre}")
print(f"Mejores parámetros: {mejores_parametros}")
print(f"Precisión en conjunto de prueba: {mejor_precision:.4f}")

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Mejor modelo en general: RandomForest
Mejores parámetros: {'max_depth': None, 'n_estimators': 200}
Precisión en conjunto de prueba: 0.7318


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

mejor_modelo = RandomForestClassifier(max_depth=None, n_estimators=200, random_state=42)
mejor_modelo.fit(X_entrenamiento, y_entrenamiento)

y_pred = mejor_modelo.predict(X_prueba)
precision = accuracy_score(y_prueba, y_pred)

print(f'Mejor modelo seleccionado: RandomForest')
print(f'Mejores parámetros: {{"max_depth": None, "n_estimators": 200}}')
print(f'Precisión en el conjunto de prueba: {precision:.4f}')

c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mejor modelo seleccionado: RandomForest
Mejores parámetros: {"max_depth": None, "n_estimators": 200}
Precisión en el conjunto de prueba: 0.7318


# Actividad 4

El código proporcionado carga un conjunto de datos de evaluación desde un archivo CSV llamado Taller_2_Titulacion_Evaluación.csv y utiliza el mejor modelo encontrado en actividades previas para realizar la predicción sobre las 568 observaciones. Luego, se crea un archivo clasificación_título.txt que contiene el Id de cada registro y una etiqueta "SÍ" o "NO" que indica la predicción del modelo.

El código sigue una estructura lógica y clara, desde la lectura de los datos hasta la escritura del archivo de salida. La utilización del mejor modelo encontrado en actividades previas garantiza que se esté utilizando el modelo más adecuado para realizar la predicción. El archivo de salida generado cumple con los requisitos de la actividad 4, sin encabezado y con la información solicitada.

In [18]:
import pandas as pd

archivo_evaluacion = 'Taller_2_Titulacion_Evaluación.csv'
datos_evaluacion = pd.read_csv(archivo_evaluacion, encoding='latin-1', delimiter=';')

ids_evaluacion = datos_evaluacion['Id']
X_evaluacion = datos_evaluacion.drop('Id', axis=1)
X_evaluacion_procesado = preprocesador.transform(X_evaluacion)

predicciones_evaluacion = best_model.predict(X_evaluacion_procesado)
predicciones_evaluacion_etiquetas = ['SÍ' if pred == 1 else 'NO' for pred in predicciones_evaluacion]
resultado_df = pd.DataFrame({'Id': ids_evaluacion, 'Predicción': predicciones_evaluacion_etiquetas})

archivo_salida = 'clasificación_título.txt'
resultado_df.to_csv(archivo_salida, sep=';', index=False, header=None)

print(f'Archivo "{archivo_salida}" generado exitosamente.')

Archivo "clasificación_título.txt" generado exitosamente.
